In [1]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange
from copy import deepcopy

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

In [2]:
#设置value的显示长度为200，默认为50
pd.set_option('max_colwidth',200)
#显示所有列，把行显示设置成最大
pd.set_option('display.max_columns', None)
#显示所有行，把列显示设置成最大
pd.set_option('display.max_rows', None)

In [3]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [4]:
def read_data(type_ = 'train', fold = None):
    if type_ != 'train' and type_ != 'val':
        data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/{}_set.csv'.format(type_), index_col = 0)
    elif type_ == 'train':
        data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/train_data_fold{}.csv'.format(fold), index_col = 0)
    elif type_ == 'val':
        data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/val_data_fold{}.csv'.format(fold), index_col = 0)
    return data

In [5]:
new_type = 'external'
ori_type = 'experimental'
data = read_data(new_type, None)
print('{} set Shape: '.format(new_type), data.shape)
data.head()

external set Shape:  (103865, 5)


,peptide,length,HLA,label,HLA_sequence
0,SDKYGLGY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
1,QLKEYLFY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
2,VFDEADRM,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
3,FLDENVHF,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY
4,VFDVGALL,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY


In [6]:
methods = ['ann', 
           'comblib_sidney2008', 
           'consensus', 
           'netmhccons', 
           'netmhcpan_ba', 
           'netmhcpan_el', 
           'netmhcstabpan', 
           'pickpocket', 
           'smm', 
           'smmpmbec',
           'anthem',
           'transformer']

In [7]:
hla_methods_dict = {}
for method in methods[:-2]:
    hlas_filename = '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/api_provide_hlas/' + method + '.hlas'
    with open(hlas_filename, 'r') as f:
        hlas = f.readlines()
        hlas = [item.replace('\n', '') for item in hlas[1:-3]]
    hla_methods_dict[method] = hlas
    print('Method = {} | # HLAs = {}'.format(method, len(hlas)))

Method = ann | # HLAs = 89
Method = comblib_sidney2008 | # HLAs = 14
Method = consensus | # HLAs = 81
Method = netmhccons | # HLAs = 2915
Method = netmhcpan_ba | # HLAs = 10386
Method = netmhcpan_el | # HLAs = 10386
Method = netmhcstabpan | # HLAs = 2915
Method = pickpocket | # HLAs = 2924
Method = smm | # HLAs = 81
Method = smmpmbec | # HLAs = 83


# Transformer的结果

In [8]:
pHLAIformer_metrics_length_hla_pd_dict = np.load('./results/pHLAIformer_metrics_length_hla_pd_dict.npy', 
                                                 allow_pickle = True).item()[new_type]
pHLAIformer_metrics_pd = pHLAIformer_metrics_length_hla_pd_dict['all']
pHLAIformer_metrics_pd

,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr
0,0.950488,0.878429,0.765103,0.868986,0.805556,0.951447,0.943259,0.805556,0.955213


In [10]:
pHLAIformer_data = np.load('./results/pHLAIformer_data_res_pd_dict.npy', allow_pickle = True).item()[new_type]
pHLAIformer_data.head()

,peptide,length,HLA,label,HLA_sequence,y_true,y_pred,y_prob
0,SDKYGLGY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.999444
1,QLKEYLFY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.737303
2,VFDEADRM,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.999999
3,FLDENVHF,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000
4,VFDVGALL,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000


# Anthem的结果

In [11]:
anthem_results_pd = pd.read_csv('/home/chujunyi/5_ZY_MHC/benchmark_methods_api/Anthem/Anthem_' + ori_type + '_res_pd.csv', index_col = 0)
anthem_results_pd = pd.merge(pHLAIformer_data, anthem_results_pd, how = 'inner', on = ['peptide', 'HLA', 'length']).reset_index(drop = True)
print('Anthem shape: ', anthem_results_pd.shape)
anthem_results_pd.head()

Anthem shape:  (97383, 10)


,peptide,length,HLA,label,HLA_sequence,y_true,y_pred,y_prob,y_score_pred,score
0,SDKYGLGY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.999444,1,0.967
1,QLKEYLFY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.737303,1,0.973
2,VFDEADRM,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.999999,0,0.920
3,FLDENVHF,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000,1,0.999
4,VFDVGALL,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000,0,0.865


# 结果处理

In [13]:
def performances(y_true, y_pred, y_prob = None, rank_or_ic50 = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    mcc = ((tp*tn) - (fn*fp)) / np.sqrt(np.float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    try:
        recall = tp / (tp+fn)
    except:
        recall = 0
        
    try:
        precision = tp / (tp+fp)
    except:
        precision = 0
        
    try: 
        f1 = 2*precision*recall / (precision+recall)
    except:
        f1 = 0
    
    if y_prob is None:
        roc_auc, aupr = np.nan, np.nan
    else:
        if rank_or_ic50:
            roc_auc = roc_auc_score(1 - y_true, y_prob)
            prec, reca, _ = precision_recall_curve(1 - y_true, y_prob)
        else:
            roc_auc = roc_auc_score(y_true, y_prob)
            prec, reca, _ = precision_recall_curve(y_true, y_prob)
            
        aupr = auc(reca, prec)

    print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
    print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
    print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
    print('auc={:.4f}|sensitivity={:.4f}|specificity={:.4f}|acc={:.4f}|mcc={:.4f}'.format(roc_auc, sensitivity, specificity, accuracy, mcc))
    print('precision={:.4f}|recall={:.4f}|f1={:.4f}|aupr={:.4f}'.format(precision, recall, f1, aupr))

    return roc_auc, accuracy, mcc, f1, sensitivity, specificity, precision, recall, aupr

In [14]:
def benchmark_results_pd(pwd, filenames):
    error_files = []
    for idx, filename in enumerate(filenames):
        if idx == 0:
            benchmark_pd = pd.read_csv(pwd + filename, sep = '\t')
            num_col = benchmark_pd.shape[1]
        else:
            try:
                temp = pd.read_csv(pwd + filename, sep = '\t')
            except:
                error_files.append(filename)
                continue

            if temp.shape[1] != num_col: 
                error_files.append(filename)
                continue
            else:
                benchmark_pd = pd.concat([benchmark_pd, temp], axis = 0)
    try:
        assert (benchmark_pd.start == 1).all()
    except:
        print(error_files)
    benchmark_pd.reset_index(drop = True, inplace = True)
    benchmark_pd.drop(columns = ['seq_num', 'start', 'end'], inplace = True)
    benchmark_pd.rename(columns = {'allele':'HLA'}, inplace = True)
    return benchmark_pd, error_files

In [15]:
def merge_benchmark_with_original_data(eval_data, hla_methods_dict, ori_type = 'experimental', method = 'ann'):
    pwd = '/home/chujunyi/5_ZY_MHC/benchmark_methods_api/output_file/' + method + '/'
    filenames = sorted(os.listdir(pwd))
    filenames = [name for name in filenames if ori_type in name]
    filenames = [item for item in filenames if item.split('_')[2] in hla_methods_dict[method]]
    if method == 'smm' or method == 'smmpmbec':
        filenames = [item for item in filenames if ('length12' not in item) and ('length13' not in item) and ('length14' not in item)]
    if method == 'comblib_sidney2008':
        filenames = [item for item in filenames if 'length9' in item]

    benchmark_pd, error_files = benchmark_results_pd(pwd, filenames)
    merged_data = pd.merge(eval_data, benchmark_pd, on = ['peptide', 'HLA', 'length'])
    
    if 'consensus_percentile_rank' in merged_data.columns: 
        merged_data.rename(columns = {'consensus_percentile_rank':'percentile_rank'}, inplace = True)
    return merged_data, error_files

In [16]:
def benchmark_label_results(merged_data, method = 'ann'):
    columns = merged_data.columns
    if 'ic50' in columns: 
        print('-----Performances according to IC50 threshold-----')
        merged_data['y_pred_ic50'] = [[0, 1][ic50 <= 500] for ic50 in merged_data.ic50]
        performance = performances(merged_data.label, merged_data.y_pred_ic50, merged_data.ic50, rank_or_ic50 = True)

    elif 'score' in columns: 
        print('-----Performances according to Score threshold-----')
        merged_data['y_pred_score'] = [[0, 1][score >= 0.5] for score in merged_data.score]
        performance = performances(merged_data.label, merged_data.y_pred_score, merged_data.score, rank_or_ic50 = False)
    
    elif 'percentile_rank' in columns: 
        print('-----Performances according to Rank threshold-----')
        merged_data['y_pred_rank'] = [[0, 1][r <= 2] for r in merged_data.percentile_rank]
        performance = performances(merged_data.label, merged_data.y_pred_rank, merged_data.percentile_rank, rank_or_ic50 = True)
    
    return merged_data, performance

In [17]:
def eval_metrics_length_hla_dict(data_eval, lengths, metrics_type):

    rank_or_ic50 = [True, False][metrics_type[0] == 'score']
    
    pHLAIformer_metrics_length_hla_dict = {}
    metrics_length_hla_dict = {}
    for length in lengths:
        data_length = data_eval[data_eval.length == length]
        hlas_length = list(set(data_length.HLA))
        pHLAIformer_metrics_length_hla_dict[length], metrics_length_hla_dict[length] = {}, {}
        for hla in tqdm(hlas_length):
            data_length_hla = data_length[data_length.HLA == hla]
            data_num = data_length_hla.shape[0]
            metrics_length_hla = performances(data_length_hla.label, 
                                              data_length_hla[metrics_type[1]], 
                                              data_length_hla[metrics_type[0]], 
                                              rank_or_ic50)
            pHLAIformer_length_hla = performances(data_length_hla.y_true, 
                                                  data_length_hla.y_pred, 
                                                  data_length_hla.y_prob, 
                                                  False)
            metrics_length_hla_dict[length][hla] = list(metrics_length_hla) + [data_num]
            pHLAIformer_metrics_length_hla_dict[length][hla] = list(pHLAIformer_length_hla) + [data_num]
    return metrics_length_hla_dict, pHLAIformer_metrics_length_hla_dict

def eval_performances_to_pd(metrics_length_dict, length):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr', 'num']
    
    performances_pd = pd.DataFrame(metrics_length_dict[length], index = metrics_name).T
    performances_pd.loc['mean'] = performances_pd.mean(axis = 0)
    performances_pd.loc['median'] = performances_pd.median(axis = 0)
    performances_pd.loc['std'] = performances_pd.std(axis = 0)
    performances_pd.loc['sum', 'num'] = performances_pd['num'].sum(axis = 0)
    performances_pd['num'] = performances_pd['num'].astype('int')
    
    return performances_pd

# 性能评估

In [18]:
merged_data_dict, error_files_dict = {}, {}
for method in tqdm(methods[:-2]):
    merged_data_dict[method], error_files_dict[method] = merge_benchmark_with_original_data(pHLAIformer_data, hla_methods_dict, ori_type, method)

for k, v in error_files_dict.items():
    print(k, v)

100%|██████████| 10/10 [00:06<00:00,  1.47it/s]

ann []
comblib_sidney2008 []
consensus []
netmhccons []
netmhcpan_ba []
netmhcpan_el []
netmhcstabpan []
pickpocket []
smm []
smmpmbec []


In [21]:
merged_data_dict['anthem'] = anthem_results_pd
merged_data_dict['anthem'].head()

,peptide,length,HLA,label,HLA_sequence,y_true,y_pred,y_prob,y_score_pred,score
0,SDKYGLGY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.999444,1,0.967
1,QLKEYLFY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.737303,1,0.973
2,VFDEADRM,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.999999,0,0.920
3,FLDENVHF,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000,1,0.999
4,VFDVGALL,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000,0,0.865


In [22]:
merged_data_dict['ann'].head()

,peptide,length,HLA,label,HLA_sequence,y_true,y_pred,y_prob,ic50,percentile_rank
0,SDKYGLGY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.999444,49.34,0.11
1,QLKEYLFY,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.737303,13305.24,5.60
2,VFDEADRM,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,0.999999,36087.13,73.00
3,FLDENVHF,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000,10629.42,4.40
4,VFDVGALL,8,HLA-A*01:01,1,YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGY,1,1,1.000000,25824.23,23.00


# 总的平均结果

In [23]:
methods_performances_dict = {}
pHLAIformer_according_methods_performances_dict = {}
for method in (methods[:-1]):
    print('*** Type = {} | Method = {} ***'.format(ori_type, method))
    merged_data = merged_data_dict[method]
    pHLAIformer_according_methods_performances_dict[method] = performances(merged_data.y_true, merged_data.y_pred, merged_data.y_prob, rank_or_ic50 = False)
    merged_data, method_performance = benchmark_label_results(merged_data, method)
    methods_performances_dict[method] = method_performance

metrics = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr']
methods_performances_pd = pd.DataFrame(methods_performances_dict, index = metrics).T
pHLAIformer_according_methods_performances_pd = pd.DataFrame(pHLAIformer_according_methods_performances_dict, index = metrics).T

methods_performances_pd.loc['pHLAIformer'] =  pHLAIformer_metrics_pd.loc[0]

*** Type = experimental | Method = ann ***
tn = 49362, fp = 2519, fn = 10108, tp = 41876
y_pred: 0 = 59470 | 1 = 44395
y_true: 0 = 51881 | 1 = 51984
auc=0.9505|sensitivity=0.8056|specificity=0.9514|acc=0.8784|mcc=0.7651
precision=0.9433|recall=0.8056|f1=0.8690|aupr=0.9552
-----Performances according to IC50 threshold-----
tn = 51348, fp = 533, fn = 27403, tp = 24581
y_pred: 0 = 78751 | 1 = 25114
y_true: 0 = 51881 | 1 = 51984
auc=0.9264|sensitivity=0.4729|specificity=0.9897|acc=0.7310|mcc=0.5402
precision=0.9788|recall=0.4729|f1=0.6377|aupr=0.9051
*** Type = experimental | Method = comblib_sidney2008 ***
tn = 10830, fp = 574, fn = 1746, tp = 9708
y_pred: 0 = 12576 | 1 = 10282
y_true: 0 = 11404 | 1 = 11454
auc=0.9602|sensitivity=0.8476|specificity=0.9497|acc=0.8985|mcc=0.8013
precision=0.9442|recall=0.8476|f1=0.8933|aupr=0.9613
-----Performances according to IC50 threshold-----
tn = 0, fp = 11404, fn = 0, tp = 11454
y_pred: 0 = 0 | 1 = 22858
y_true: 0 = 11404 | 1 = 11454
auc=0.9059|sensi

In [24]:
pHLAIformer_according_methods_performances_pd.T

,ann,comblib_sidney2008,consensus,netmhccons,netmhcpan_ba,netmhcpan_el,netmhcstabpan,pickpocket,smm,smmpmbec,anthem
roc_auc,0.950488,0.960245,0.950488,0.950488,0.950488,0.950488,0.950488,0.950488,0.959452,0.959452,0.958065
accuracy,0.878429,0.898504,0.878429,0.878429,0.878429,0.878429,0.878429,0.878429,0.893614,0.893614,0.891080
mcc,0.765103,0.801274,0.765103,0.765103,0.765103,0.765103,0.765103,0.765103,0.792973,0.792973,0.788021
f1,0.868986,0.893265,0.868986,0.868986,0.868986,0.868986,0.868986,0.868986,0.886960,0.886960,0.884166
sensitivity,0.805556,0.847564,0.805556,0.805556,0.805556,0.805556,0.805556,0.805556,0.833850,0.833850,0.830519
specificity,0.951447,0.949667,0.951447,0.951447,0.951447,0.951447,0.951447,0.951447,0.953507,0.953507,0.951768
precision,0.943259,0.944174,0.943259,0.943259,0.943259,0.943259,0.943259,0.943259,0.947296,0.947296,0.945223
recall,0.805556,0.847564,0.805556,0.805556,0.805556,0.805556,0.805556,0.805556,0.833850,0.833850,0.830519
aupr,0.955213,0.961306,0.955213,0.955213,0.955213,0.955213,0.955213,0.955213,0.962464,0.962464,0.961248


In [25]:
methods_performances_pd.T

,ann,comblib_sidney2008,consensus,netmhccons,netmhcpan_ba,netmhcpan_el,netmhcstabpan,pickpocket,smm,smmpmbec,anthem,pHLAIformer
roc_auc,0.926425,0.905888,0.920295,0.933087,0.934638,0.941289,0.903607,0.909781,0.901053,0.907309,0.950246,0.950488
accuracy,0.731035,0.501094,0.732663,0.735878,0.734646,0.733096,0.703981,0.610234,0.722034,0.708407,0.895505,0.878429
mcc,0.540186,NaN,0.541367,0.549441,0.547557,0.548270,0.485653,0.342473,0.516039,0.499291,0.794424,0.765103
f1,0.637656,0.667638,0.641517,0.645454,0.643182,0.638396,0.594540,0.370140,0.627762,0.598456,0.890569,0.868986
sensitivity,0.472857,1.000000,0.477936,0.480359,0.477839,0.470741,0.433633,0.228820,0.468271,0.434117,0.849496,0.805556
specificity,0.989726,0.000000,0.987895,0.991905,0.991962,0.995972,0.974866,0.992406,0.976347,0.983293,0.941612,0.951447
precision,0.978777,0.501094,0.975346,0.983459,0.983490,0.991532,0.945316,0.967939,0.952016,0.963018,0.935815,0.943259
recall,0.472857,1.000000,0.477936,0.480359,0.477839,0.470741,0.433633,0.228820,0.468271,0.434117,0.849496,0.805556
aupr,0.905118,0.895950,0.899319,0.913248,0.915410,0.953607,0.901843,0.905351,0.890218,0.892781,0.959289,0.955213


In [26]:
pHLAIformer_according_methods_performances_pd - methods_performances_pd

,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr
ann,0.024062,0.147393,0.224918,0.231330,0.332699,-0.038280,-0.035517,0.332699,0.050095
anthem,0.007819,-0.004426,-0.006403,-0.006402,-0.018977,0.010156,0.009408,-0.018977,0.001959
comblib_sidney2008,0.054357,0.397410,NaN,0.225626,-0.152436,0.949667,0.443081,-0.152436,0.065357
consensus,0.030192,0.145766,0.223737,0.227469,0.327620,-0.036449,-0.032087,0.327620,0.055894
netmhccons,0.017400,0.142550,0.215663,0.223532,0.325196,-0.040458,-0.040199,0.325196,0.041965
netmhcpan_ba,0.015850,0.143783,0.217546,0.225804,0.327716,-0.040516,-0.040230,0.327716,0.039803
netmhcpan_el,0.009199,0.145333,0.216833,0.230590,0.334815,-0.044525,-0.048272,0.334815,0.001606
netmhcstabpan,0.046881,0.174448,0.279451,0.274446,0.371922,-0.023419,-0.002056,0.371922,0.053370
pHLAIformer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pickpocket,0.040706,0.268194,0.422630,0.498846,0.576735,-0.040959,-0.024679,0.576735,0.049862


# 针对每个HLA的结果

In [29]:
pHLAIformer_methods_metrics_length_hla_dict, methods_metrics_length_hla_dict = {}, {}
for method in methods[:-1]:
    pHLAIformer_methods_metrics_length_hla_dict[method], methods_metrics_length_hla_dict[method] = {}, {}
    
    if method == 'smm' or method == 'smmpmbec':
        lengths = list(range(8, 12))
    elif method == 'comblib_sidney2008':
        lengths = [9]
    else:
        lengths = list(range(8, 15))
    
    if 'ic50' and 'y_pred_ic50' in merged_data_dict[method].columns: 
        m = ['ic50', 'y_pred_ic50']
    elif 'percentile_rank' and 'y_pred_rank' in merged_data_dict[method].columns: 
        m = ['percentile_rank', 'y_pred_rank']
    elif 'score' and 'y_pred_score' in merged_data_dict[method].columns: 
        m = ['score', 'y_pred_score']
     
    metrics_length_hla_dict, pHLAIformer_metrics_length_hla_dict = eval_metrics_length_hla_dict(merged_data_dict[method], lengths, m)
    pHLAIformer_methods_metrics_length_hla_dict[method][m[0]], methods_metrics_length_hla_dict[method][m[0]] = {}, {}
    for l in lengths:
        methods_metrics_length_hla_dict[method][m[0]][l]  = eval_performances_to_pd(metrics_length_hla_dict, l)
        pHLAIformer_methods_metrics_length_hla_dict[method][m[0]][l] = eval_performances_to_pd(pHLAIformer_metrics_length_hla_dict, l)

  0%|          | 0/5 [00:00<?, ?it/s]

tn = 337, fp = 2, fn = 305, tp = 34
y_pred: 0 = 642 | 1 = 36
y_true: 0 = 339 | 1 = 339
auc=0.8320|sensitivity=0.1003|specificity=0.9941|acc=0.5472|mcc=0.2105
precision=0.9444|recall=0.1003|f1=0.1813|aupr=0.8297
tn = 307, fp = 32, fn = 146, tp = 193
y_pred: 0 = 453 | 1 = 225
y_true: 0 = 339 | 1 = 339
auc=0.8612|sensitivity=0.5693|specificity=0.9056|acc=0.7375|mcc=0.5043
precision=0.8578|recall=0.5693|f1=0.6844|aupr=0.8641
tn = 7504, fp = 74, fn = 4873, tp = 2715
y_pred: 0 = 12377 | 1 = 2789
y_true: 0 = 7578 | 1 = 7588
auc=0.9578|sensitivity=0.3578|specificity=0.9902|acc=0.6738|mcc=0.4492
precision=0.9735|recall=0.3578|f1=0.5233|aupr=0.9596
tn = 7300, fp = 278, fn = 998, tp = 6590
y_pred: 0 = 8298 | 1 = 6868
y_true: 0 = 7578 | 1 = 7588
auc=0.9760|sensitivity=0.8685|specificity=0.9633|acc=0.9159|mcc=0.8355
precision=0.9595|recall=0.8685|f1=0.9117|aupr=0.9742
tn = 579, fp = 0, fn = 533, tp = 45
y_pred: 0 = 1112 | 1 = 45
y_true: 0 = 579 | 1 = 578
auc=0.9158|sensitivity=0.0779|specificity=1.

 80%|████████  | 4/5 [00:00<00:00, 17.10it/s]

tn = 2337, fp = 91, fn = 913, tp = 1534
y_pred: 0 = 3250 | 1 = 1625
y_true: 0 = 2428 | 1 = 2447
auc=0.9177|sensitivity=0.6269|specificity=0.9625|acc=0.7941|mcc=0.6252
precision=0.9440|recall=0.6269|f1=0.7534|aupr=0.9150
tn = 2218, fp = 210, fn = 390, tp = 2057
y_pred: 0 = 2608 | 1 = 2267
y_true: 0 = 2428 | 1 = 2447
auc=0.9454|sensitivity=0.8406|specificity=0.9135|acc=0.8769|mcc=0.7560
precision=0.9074|recall=0.8406|f1=0.8727|aupr=0.9417
tn = 8852, fp = 124, fn = 4146, tp = 4861
y_pred: 0 = 12998 | 1 = 4985
y_true: 0 = 8976 | 1 = 9007
auc=0.9486|sensitivity=0.5397|specificity=0.9862|acc=0.7626|mcc=0.5874
precision=0.9751|recall=0.5397|f1=0.6948|aupr=0.9390
tn = 8612, fp = 364, fn = 1356, tp = 7651
y_pred: 0 = 9968 | 1 = 8015
y_true: 0 = 8976 | 1 = 9007
auc=0.9657|sensitivity=0.8495|specificity=0.9594|acc=0.9044|mcc=0.8137
precision=0.9546|recall=0.8495|f1=0.8990|aupr=0.9672
tn = 2223, fp = 13, fn = 1096, tp = 1149
y_pred: 0 = 3319 | 1 = 1162
y_true: 0 = 2236 | 1 = 2245
auc=0.9480|sensit

  0%|          | 0/5 [00:00<?, ?it/s]

...

tn = 2218, fp = 210, fn = 390, tp = 2057
y_pred: 0 = 2608 | 1 = 2267
y_true: 0 = 2428 | 1 = 2447
auc=0.9454|sensitivity=0.8406|specificity=0.9135|acc=0.8769|mcc=0.7560
precision=0.9074|recall=0.8406|f1=0.8727|aupr=0.9417
tn = 8372, fp = 604, fn = 972, tp = 8035
y_pred: 0 = 9344 | 1 = 8639
y_true: 0 = 8976 | 1 = 9007
auc=0.9640|sensitivity=0.8921|specificity=0.9327|acc=0.9124|mcc=0.8254
precision=0.9301|recall=0.8921|f1=0.9107|aupr=0.9691
tn = 8612, fp = 364, fn = 1356, tp = 7651
y_pred: 0 = 9968 | 1 = 8015
y_true: 0 = 8976 | 1 = 9007
auc=0.9657|sensitivity=0.8495|specificity=0.9594|acc=0.9044|mcc=0.8137
precision=0.9546|recall=0.8495|f1=0.8990|aupr=0.9672
tn = 2107, fp = 129, fn = 297, tp = 1948
y_pred: 0 = 2404 | 1 = 2077
y_true: 0 = 2236 | 1 = 2245
auc=0.9684|sensitivity=0.8677|specificity=0.9423|acc=0.9049|mcc=0.8122
precision=0.9379|recall=0.8677|f1=0.9014|aupr=0.9716
tn = 2078, fp = 158, fn = 291, tp = 1954
y_pred: 0 = 2369 | 1 = 2112
y_true: 0 = 2236 | 1 = 2245
auc=0.9619|sensiti

100%|██████████| 4/4 [00:00<00:00, 93.50it/s]

tn = 1503, fp = 67, fn = 325, tp = 1248
y_pred: 0 = 1828 | 1 = 1315
y_true: 0 = 1570 | 1 = 1573
auc=0.9653|sensitivity=0.7934|specificity=0.9573|acc=0.8753|mcc=0.7609
precision=0.9490|recall=0.7934|f1=0.8643|aupr=0.9658
tn = 697, fp = 126, fn = 104, tp = 717
y_pred: 0 = 801 | 1 = 843
y_true: 0 = 823 | 1 = 821
auc=0.9347|sensitivity=0.8733|specificity=0.8469|acc=0.8601|mcc=0.7205
precision=0.8505|recall=0.8733|f1=0.8618|aupr=0.9417
tn = 747, fp = 76, fn = 169, tp = 652
y_pred: 0 = 916 | 1 = 728
y_true: 0 = 823 | 1 = 821
auc=0.9276|sensitivity=0.7942|specificity=0.9077|acc=0.8510|mcc=0.7064
precision=0.8956|recall=0.7942|f1=0.8418|aupr=0.9305
tn = 1046, fp = 146, fn = 284, tp = 907
y_pred: 0 = 1330 | 1 = 1053
y_true: 0 = 1192 | 1 = 1191
auc=0.8849|sensitivity=0.7615|specificity=0.8775|acc=0.8196|mcc=0.6434
precision=0.8613|recall=0.7615|f1=0.8084|aupr=0.8957
tn = 1100, fp = 92, fn = 302, tp = 889
y_pred: 0 = 1402 | 1 = 981
y_true: 0 = 1192 | 1 = 1191
auc=0.9130|sensitivity=0.7464|specifi


100%|██████████| 3/3 [00:00<00:00, 105.06it/s]


tn = 78, fp = 9, fn = 33, tp = 54
y_pred: 0 = 111 | 1 = 63
y_true: 0 = 87 | 1 = 87
auc=0.8119|sensitivity=0.6207|specificity=0.8966|acc=0.7586|mcc=0.5381
precision=0.8571|recall=0.6207|f1=0.7200|aupr=0.8658
tn = 79, fp = 8, fn = 33, tp = 54
y_pred: 0 = 112 | 1 = 62
y_true: 0 = 87 | 1 = 87
auc=0.8197|sensitivity=0.6207|specificity=0.9080|acc=0.7644|mcc=0.5520
precision=0.8710|recall=0.6207|f1=0.7248|aupr=0.8683
tn = 107, fp = 10, fn = 44, tp = 73
y_pred: 0 = 151 | 1 = 83
y_true: 0 = 117 | 1 = 117
auc=0.8054|sensitivity=0.6239|specificity=0.9145|acc=0.7692|mcc=0.5627
precision=0.8795|recall=0.6239|f1=0.7300|aupr=0.8561
tn = 98, fp = 19, fn = 45, tp = 72
y_pred: 0 = 143 | 1 = 91
y_true: 0 = 117 | 1 = 117
auc=0.7895|sensitivity=0.6154|specificity=0.8376|acc=0.7265|mcc=0.4646
precision=0.7912|recall=0.6154|f1=0.6923|aupr=0.8297
tn = 308, fp = 9, fn = 46, tp = 270
y_pred: 0 = 354 | 1 = 279
y_true: 0 = 317 | 1 = 316
auc=0.9387|sensitivity=0.8544|specificity=0.9716|acc=0.9131|mcc=0.8319
precis

In [30]:
length_methods_dict = {}
for length in range(8,15):
    if length == 9: 
        length_methods_dict[length] = ['pHLAIformer', 'ann', 'consensus', 'netmhccons', 'netmhcpan_ba', 'netmhcpan_el', 'netmhcstabpan', 'pickpocket', 'smm', 'smmpmbec', 'anthem', 'comblib_sidney2008']
    elif length in range(12,15): 
        length_methods_dict[length] = ['pHLAIformer', 'ann', 'consensus', 'netmhccons', 'netmhcpan_ba', 'netmhcpan_el', 'netmhcstabpan', 'pickpocket', 'anthem']
    else: 
        length_methods_dict[length] = ['pHLAIformer',  'ann', 'consensus', 'netmhccons', 'netmhcpan_ba', 'netmhcpan_el', 'netmhcstabpan', 'pickpocket', 'smm', 'smmpmbec', 'anthem']

In [31]:
length_methods_newname_dict = {}
for length in range(8,15):
    if length == 9: 
        length_methods_newname_dict[length] = ['pHLAIformer', 'ANN', 'Consensus', 'NetMHCcons', 'NetMHCpan_BA', 'NetMHCpan_EL', 'NetMHCstabpan', 'PickPocket', 'SMM', 'SMMPMBEC', 'Anthem', 'CombLib']
    elif length in range(12,15): 
        length_methods_newname_dict[length] = ['pHLAIformer', 'ANN', 'Consensus', 'NetMHCcons', 'NetMHCpan_BA', 'NetMHCpan_EL', 'NetMHCstabpan', 'PickPocket', 'Anthem']
    else: 
        length_methods_newname_dict[length] = ['pHLAIformer', 'ANN', 'Consensus', 'NetMHCcons', 'NetMHCpan_BA', 'NetMHCpan_EL', 'NetMHCstabpan', 'PickPocket', 'SMM', 'SMMPMBEC', 'Anthem']

In [61]:
cols = ['roc_auc', 'accuracy', 'mcc', 'f1']
length, col = 14, cols[3]

length_methods = length_methods_dict[length]
length_methods_newname = length_methods_newname_dict[length]
res_pd = pHLAIformer_metrics_length_hla_pd_dict[length][col]
for method in length_methods[1:]:
    if 'ic50' in methods_metrics_length_hla_dict[method].keys(): 
        m = 'ic50'
    elif 'percentile_rank' in methods_metrics_length_hla_dict[method].keys(): 
        m = 'percentile_rank'
    elif 'score' in methods_metrics_length_hla_dict[method].keys(): 
        m = 'score'
        
    if method != 'anthem':
        temp = methods_metrics_length_hla_dict[method][m][length][col]
    else:
        temp = methods_metrics_length_hla_dict[method][m][length][col]
    res_pd = pd.concat([res_pd, temp], axis = 1)
res_pd.columns = length_methods_newname
print([res_pd[col].count()-3 for col in res_pd.columns])
print(res_pd.iloc[:-4, :].min().min(), res_pd.iloc[:-4, :].max().max())
res_pd.iloc[:-4, :]

[5, 5, 5, 5, 5, 5, 5, 5, 3]
0.0 0.8466257668711655


,pHLAIformer,ANN,Consensus,NetMHCcons,NetMHCpan_BA,NetMHCpan_EL,NetMHCstabpan,PickPocket,Anthem
HLA-B*08:01,0.121406,0.123711,0.159468,0.022140,0.043796,0.000000,0.014815,0.007435,NaN
HLA-B*18:01,0.296296,0.161926,0.243299,0.174672,0.051643,0.000000,0.241449,0.060748,NaN
HLA-A*24:02,0.638889,0.260870,0.326531,0.360000,0.181818,0.048780,0.452830,0.291667,0.593750
HLA-A*01:01,0.846626,0.641791,0.796117,0.656934,0.464135,0.453782,0.743243,0.104167,0.845679
HLA-A*02:01,0.526316,0.188679,0.153846,0.310345,0.254545,0.000000,0.406250,0.360656,0.556962
